# Handling Subprocess Errors

- External commands can fail in multiple ways: non-zero exit codes, missing executables, or hanging processes.  
- Using `subprocess.run(..., check=True)` shifts return-code checks into exceptions you can catch.  
- Specific exception types (`CalledProcessError`, `FileNotFoundError`, `TimeoutExpired`) let you distinguish failure modes and respond appropriately.

## subprocess.CalledProcessError Attributes

- `e.returncode`: the non-zero exit status of the command.  
- `e.cmd`: the exact command invoked (list or string form).  
- `e.stdout` / `e.output`: captured standard output, if `capture_output=True`.  
- `e.stderr`: captured standard error, if `capture_output=True`.  
- These attributes let you log or display detailed diagnostics when a command fails.  

In [1]:
import subprocess

cmd = ["ls", "missing_dir"]

try:
    subprocess.run(cmd, check=True, capture_output=True, text=True)
except subprocess.CalledProcessError as err:
    print(f"Command executed: {err.cmd}")
    print(f"Return code {err.returncode}")
    print(f"STDOUT capture: {err.stdout}")
    print(f"STDERR capture: {err.stderr}")

Command executed: ['ls', 'missing_dir']
Return code 1
STDOUT capture: 
STDERR capture: ls: missing_dir: No such file or directory



## Handling FileNotFoundError

- If the executable itself isn’t in `PATH`, `subprocess.run()` raises `FileNotFoundError` before running.  
- Catching it separately lets you inform the user that a required tool isn’t installed, rather than treating it as a generic failure.  

In [11]:
import subprocess

cmd = ["fakecmd", "--version"]

try:
    subprocess.run(cmd, check=True, capture_output=True, text=True)
except FileNotFoundError as err:
    print("FileNotFoundError caught!")
    print(f"  The command '{cmd[0]}' was not found on this system.")

FileNotFoundError caught!
  The command 'fakecmd' was not found on this system.


## Handling subprocess.TimeoutExpired

- Adding `timeout=<seconds>` to `subprocess.run()` kills the process if it runs too long.  
- A `TimeoutExpired` exception is raised, containing `cmd`, `timeout`, and any partial `stdout`/`stderr`.  
- Use this to prevent hung scripts and to implement retry or fallback logic.  

In [5]:
import subprocess

cmd = ["sleep", "5"]

try:
    subprocess.run(cmd, timeout=2, capture_output=True, text=True)
    print("Command completed within timeout.")
except subprocess.TimeoutExpired as err:
    print("TimeoutExpired caught!")
    print(f"  Command: {err.cmd}")
    print(f"  Timeout after {err.timeout} seconds")

TimeoutExpired caught!
  Command: ['sleep', '5']
  Timeout after 2 seconds


## Recommended Error Handling Strategy

- Wrap `subprocess.run()` in a `try` block.  
- First catch `FileNotFoundError` to detect missing executables.  
- Next catch `subprocess.TimeoutExpired` if you use timeouts.  
- Then catch `subprocess.CalledProcessError` for non-zero exits.  
- Finally, if necessary, an `except Exception` block can log any other unexpected issues.  
- This layered approach keeps your script robust and your errors informative.  